# P3. OpenStreetMap Case Study - Milwaukee, WI
## Map Area
Milwaukee, WI, United States
- https://www.openstreetmap.org/relation/251075
- https://s3.amazonaws.com/metro-extracts.mapzen.com/milwaukee_wisconsin.osm.bz2

I selected Milwaukee because it is has been recommended to me by some friends recently as an interesting and underrated city to visit. I decided to use it for this project to see what the city has to offer, in preparation for a future visit.

## Problems Encountered
Through analysis of a small sample of the map, the data appeared to be very clean. However, on my first scan of the entire xml document, I came across a few issues with the data:
- Inconsistent street name abbreviations (ex: "Street" vs "St" vs "St."; or "Ave" vs "Ave.")
- Street names containing postal code information
- Inconsistent numeric postal code formats ("XXXXX" vs "XXXXX-XXXX")
- Postal codes containing non-numeric information (ex. "WI" as value of postal code)

## Auditing Data and Determining Necessary Cleaning Procedures

In [2]:
# Import packages needed for xml parsing and data printing
import xml.etree.ElementTree as ET
import pprint

In [17]:
# Set file name
# OSM_FILE = 'milwaukee_wisconsin.osm'
OSM_FILE = 'milwaukee_sample.osm'

### Count number of tags in selected OSM xml file

In [18]:
def childTags(parent, tags={}):
    '''Return all child tags within parent tag
    
    parent -- name of parent tag
    '''
    for child in parent:
        tags = childTags(child, tags)
    if parent.tag in tags:
        tags[parent.tag] += 1
    else:
        tags[parent.tag] = 1
    return tags

def count_tags(filename):
    '''Return number of each tag found in xml file
    
    filename -- XML File Name (string)
    '''
    tree = ET.parse(filename)
    root = tree.getroot()
    tags = {}
    tags = childTags(root)
    return tags

tags = count_tags(OSM_FILE)
pprint.pprint(tags)

{'bounds': 1,
 'member': 9972,
 'nd': 1039528,
 'node': 840763,
 'osm': 1,
 'relation': 996,
 'tag': 519564,
 'way': 94038}


### Audit street names and postal codes for unexpected formats

In [19]:
# Look at street names for unexpected issues
import re
from collections import defaultdict
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected = ['Street', 'Avenue', 'Boulevard', 'Drive', 'Court', 'Lane', 'Place', 'Road', 
            'Parkway', 'Square', 'Way', 'Pointe', 'Circle', 'Trail']

def is_street_name(elem):
    '''Outputs "true" if input element attribute == "addr:street"'''
    return (elem.attrib['k'] == 'addr:street')

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

In [20]:
# Also audit issues with postcodes
postcode_re = re.compile(r'^\d{5}$')

def is_postcode_name(elem):
    return(elem.attrib['k'] == 'addr:postcode')

def audit_postcode_name(postcode_names, postcode):
    m = postcode_re.search(postcode)
    if not m:
        postcode_names[postcode].add(postcode)

In [21]:
def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    postcode_names = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events = ("start",)):
        if elem.tag == 'way':
            for tag in elem.iter('tag'):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
                elif is_postcode_name(tag):
                    audit_postcode_name(postcode_names, tag.attrib['v'])
    osm_file.close()
    return street_types, postcode_names

In [22]:
st_types, pc_names = audit(OSM_FILE)

In [23]:
# List street types / names
st_types

defaultdict(set,
            {'164': {'HWY 164', 'N6620 HWY 164', 'State Road 164'},
             '31': {'State Highway 31'},
             '41': {'Highway 41'},
             '53076': {'53076'},
             '83': {'North Highway 83'},
             'Ave': {'E Wisconsin Ave',
              'Milwaukee Ave',
              'N. Prospect Ave',
              'North Murray Ave',
              'North Summit Ave',
              'S Howell Ave',
              'Summit Ave',
              'W Appleton Ave',
              'W Fond du Lac Ave',
              'W Grand Ave',
              'W Layton Ave',
              'Wisconsin Ave'},
             'Ave.': {'E. North Ave.',
              'N. Prospect Ave.',
              'North Oakland Ave.',
              'W. Michigan Ave.'},
             'Blvd': {'N Grandview  Blvd'},
             'Ct': {'Manchester Ct', 'W Oakwood Part Ct'},
             'Dr': {'Dr Martin Luther King Dr',
              'E Capitol Dr',
              'N9581 Bancroft Dr',
              'W 

There are a number of inconsistencies in street names. Some street types are abbreviated while others aren't (ex: "Street" vs. "St"). To solve this, abbreviated street types will be mapped to their unabbreviated forms. Street names with unusual street types (for example, numeric street types) will be mapped on a case-by-case basis.

There is one street name which contained only a postal code. To avoid a chance of error by including this point, this node will be omitted from importing to our database.

In [24]:
# List unusual postal codes
pc_names

defaultdict(set,
            {'1729': {'1729'},
             '53202-2001': {'53202-2001'},
             '53203-3002': {'53203-3002'},
             '53212-3839': {'53212-3839'},
             '53212-4099': {'53212-4099'},
             '53403-9998': {'53403-9998'},
             'WI': {'WI'}})

For the database, we will try to keep the numeric format of postal codes consistent for all points. Therefore, postal codes found that do not match the common "XXXXX" numeric format are mapped to the correct format if possible or skipped if this conversion is not obvious

### Create mappings for street names and postcodes

In [25]:
mapping_street_name = {'Ave': 'Avenue', 
           'Ave.': 'Avenue',
           'Blvd': 'Boulevard',
           'Ct': 'Court',
           'Dr': 'Drive', 
           'Pkwy': 'Parkway',
           'Rd': 'Road',
           'Rd.': 'Road',
           'St': 'Street',
           'St.': 'Street'}

changes_street_name = {'HWY 164': 'Highway 164',
                       'N6620 HWY 164': 'N6620 Highway 164'}

skip_street_name = ['53076']

mapping_postcode = {'53202-2001': '53202',
                    '53203-3002': '53203',
                    '53212-3839': '53212',
                    '53212-4099': '53212',
                    '53403-9998': '53403',
                    '53217-5399': '53217',
                    '"Milwaukee WI, 53222"': '53222'}
skip_postcode = ['WI', '1729']

In [26]:
# Update street names and postcodes
def update_street_name(name, mapping):
    m = street_type_re.search(name)
    if m:
        if (m.group() == '164') and (name in mapping.keys()):
            name = mapping[name]
        else:
            name = street_type_re.sub(mapping[m.group()], name)
        return name

def update_postcode(name, mapping):
    m = postcode_re.search(name)
    if not m:
        if name in mapping.keys():
            name = mapping[name]
            return name

In [27]:
# Test update street type function
unchanged_names = []
for st_type, ways in st_types.iteritems():
    for name in ways:
        m = street_type_re.search(name)
        if m.group() in mapping_street_name.keys():
            better_name = update_street_name(name, mapping_street_name)
            print name, '=>', better_name
        elif name in changes_street_name.keys():
            better_name = update_street_name(name, changes_street_name)
            print name, '=>', better_name
        else:
            unchanged_names.append(name)
print unchanged_names

E. North Ave. => E. North Avenue
N. Prospect Ave. => N. Prospect Avenue
W. Michigan Ave. => W. Michigan Avenue
North Oakland Ave. => North Oakland Avenue
S. Jefferson St. => S. Jefferson Street
N. Weil St. => N. Weil Street
N. Commerce St. => N. Commerce Street
S 13th St => S 13th Street
N 124th St => N 124th Street
N Main St => N Main Street
E Sumner St => E Sumner Street
N Pine St => N Pine Street
N Rochester St => N Rochester Street
N Weil St => N Weil Street
W Sumner St => W Sumner Street
21st St => 21st Street
West County Line Rd => West County Line Road
N Port Washington Rd => N Port Washington Road
Golf Rd => Golf Road
N Grandview  Blvd => N Grandview  Boulevard
HWY 164 => Highway 164
N6620 HWY 164 => N6620 Highway 164
W Grand Ave => W Grand Avenue
North Murray Ave => North Murray Avenue
W Fond du Lac Ave => W Fond du Lac Avenue
W Layton Ave => W Layton Avenue
Milwaukee Ave => Milwaukee Avenue
North Summit Ave => North Summit Avenue
Summit Ave => Summit Avenue
S Howell Ave => S 

In [28]:
# Test update postcode function
for pc_name, ways in pc_names.iteritems():
    for name in ways:
        better_pc_name = update_postcode(name, mapping_postcode)
        print pc_name, '=>', better_pc_name

53403-9998 => 53403
1729 => None
WI => None
53203-3002 => 53203
53202-2001 => 53202
53212-4099 => 53212
53212-3839 => 53212


In applying these corrections, we can see from the printed results that all street names and postal codes found to be inconsistent during our audit have been either corrected, skipped, or omitted from the database

## Export (w/ changes) to CSV File
Previously defined update functions to correct street name and postal code inconsistencies have been applied to the export function, which was written as exercises throughout the SQL Data Wrangling course.

In [29]:
# Begin preparing to export cleaned data to .csv files
import csv
import codecs
import re
import xml.etree.cElementTree as ET

import cerberus

import schema

OSM_PATH = "milwaukee_wisconsin.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

SCHEMA = schema.schema

# Mappings
mapping_street_name = {'Ave': 'Avenue', 
           'Ave.': 'Avenue',
           'Blvd': 'Boulevard',
           'Ct': 'Court',
           'Dr': 'Drive', 
           'Pkwy': 'Parkway',
           'Rd': 'Road',
           'Rd.': 'Road',
           'St': 'Street',
           'St.': 'Street'}

changes_street_name = {'HWY 164': 'Highway 164',
                       'N6620 HWY 164': 'N6620 Highway 164'}

skip_street_name = ['53076']

mapping_postcode = {'53202-2001': '53202',
                    '53203-3002': '53203',
                    '53212-3839': '53212',
                    '53212-4099': '53212',
                    '53403-9998': '53403',
                    '53217-5399': '53217',
                    'Milwaukee WI, 53222': '53222'}
skip_postcode = ['WI', '1729']

# Make sure the fields order in the csvs matches the column order in the sql table schema
# Table: nodes.csv
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
# Table: nodes_tags.csv
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
# Table: ways.csv
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
# Table: ways_tags.csv
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
# Table: ways_nodes.csv
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

In [36]:
# Update street names and postcodes
def update_street_name(name, mapping):
    m = street_type_re.search(name)
    if m and m.group() in mapping.keys():
        temp_name = name
        name = re.sub(street_type_re, mapping[m.group()], name)
        # Print name changes as they occur
#        print temp_name, '=>', name
    return name

def update_postcode(name, mapping):
    m = postcode_re.search(name)
    if not m:
        if name in mapping.keys():
            name = mapping[name]
    return name

In [41]:
def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    if element.tag=='node':
        for i in node_attr_fields:
            node_attribs[i]=element.attrib[i]
            
    if element.tag=='way':
        for i in way_attr_fields:
            way_attribs[i]=element.attrib[i]
        
    for tag in element.iter("tag"):
        dic={}
        attributes=tag.attrib
        kvalue = tag.attrib['k']
        vvalue = tag.attrib['v']
        if problem_chars.search(kvalue):
            continue
        if vvalue in skip_street_name or vvalue in skip_postcode:
            continue
        if kvalue.startswith('addr:'):
            if kvalue == 'addr:street':
                vvalue_temp = vvalue
                vvalue = update_street_name(vvalue, mapping_street_name)
                if vvalue_temp != vvalue:
                    print vvalue_temp, '=>', vvalue                
            elif kvalue == 'addr:postcode':
                vvalue_temp = vvalue
                vvalue = update_postcode(vvalue, mapping_postcode)
                if vvalue_temp != vvalue:
                    print vvalue_temp, '=>', vvalue
        
        tag.attrib['k'] = kvalue
        tag.attrib['v'] = vvalue
        
        if element.tag=='node':
            dic['id']=node_attribs['id']
        else:
            dic['id']=way_attribs['id']
        dic['value']=attributes['v']
        
        colon_k=LOWER_COLON.search(tag.attrib['k'])
        if colon_k:
            index = tag.attrib['k'].find(':')
            if index > 0:
                dic['type'] = tag.attrib['k'][:index]
                dic['key'] = tag.attrib['k'][index+1:]
            
        else:
            dic['key']=attributes['k']
            dic['type']='regular'
        
        tags.append(dic)
    
    if element.tag=='way':
        position=0
        for nd in element.iter("nd"):
            way_node_dic={}
            way_node_dic['id']=way_attribs['id']
            way_node_dic['node_id']=nd.attrib['ref']
            way_node_dic['position']=position
            position = position + 1
            way_nodes.append(way_node_dic)
            
    if element.tag == 'node':
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

In [42]:
# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_strings = (
            "{0}: {1}".format(k, v if isinstance(v, str) else ", ".join(v))
            for k, v in errors.iteritems()
        )
        raise cerberus.ValidationError(
            message_string.format(field, "\n".join(error_strings))
        )


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

In [43]:
# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])

In [44]:
if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=False)

W Fairy Chasm Rd => W Fairy Chasm Road
Milwaukee WI, 53222 => 53222
North Murray Ave => North Murray Avenue
Norhardt Dr => Norhardt Drive
Norhardt Dr => Norhardt Drive
Norhardt Dr => Norhardt Drive
E Brady St => E Brady Street
W Capitol Dr => W Capitol Drive
S7959 Racine Ave => S7959 Racine Avenue
S 108th St => S 108th Street
W National Ave => W National Avenue
S 108th St => S 108th Street
Scenic Ct => Scenic Court
Anthony Ave. => Anthony Avenue
Village Square Dr => Village Square Drive
W. Wisconsin Ave. => W. Wisconsin Avenue
W. Wisconsin Ave. => W. Wisconsin Avenue
N. Main St => N. Main Street
S. Main St => S. Main Street
W North Ave => W North Avenue
W North Ave => W North Avenue
W Watertown Plank Rd => W Watertown Plank Road
N Green Bay Rd => N Green Bay Road
N Green Bay Rd => N Green Bay Road
North 51st St. => North 51st Street
N Green Bay Rd => N Green Bay Road
N Green Bay Rd => N Green Bay Road
N Green Bay Rd => N Green Bay Road
W Donges Bay Rd => W Donges Bay Road
Cedarburg Rd 

## Run Database Queries
For this project, the 'sqlite3' library was imported to allow us to query the database directly through the ipython notebook. Additionally, a couple of functions were developed to print query results in a readable format.

In [48]:
import sqlite3

def print_results(db_name, query):
    db = sqlite3.connect(db_name)
    c = db.cursor()
    c.execute(query)
    rows = c.fetchall()
    
    for row in rows:
        line = ''
        for element in row:
            if isinstance(element, basestring):
                element = element.encode('utf-8')
            else:
                element = str(element)
            line = line + element + '\t'
        print line
    db.close()

def print_results_2(db_name, query):
    db = sqlite3.connect(db_name)
    c = db.cursor()
    c.execute(query)
    rows = c.fetchall()

    for row in rows:
        print row

    db.close()

db_name = 'milwaukee_osm.db'

### Show Database Information (Schema and Tables)

In [49]:
# Show Schema and Tables
con = sqlite3.connect(db_name)
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print '----------Table Names----------'
print(cursor.fetchall())
print


query = 'SELECT * FROM sqlite_master;'
db = sqlite3.connect(db_name)
c = db.cursor()
c.execute(query)
rows = c.fetchall()

print '------------Schema-------------'
for row in rows:
    print row[0], '\t\t', row[1]
    print 'line count', '\t', row[3]
    print row[4]
    print

db.close()

----------Table Names----------
[(u'nodes',), (u'nodes_tags',), (u'ways',), (u'ways_tags',), (u'ways_nodes',)]

------------Schema-------------
table 		nodes
line count 	137034
CREATE TABLE nodes(id, lat, lon, user, uid, version, changeset, timestamp)

table 		nodes_tags
line count 	162854
CREATE TABLE nodes_tags(id, key, value, type)

table 		ways
line count 	163025
CREATE TABLE ways(id, user, uid, version, changeset, timestamp)

table 		ways_tags
line count 	163026
CREATE TABLE ways_tags(id, key, value, type)

table 		ways_nodes
line count 	163027
CREATE TABLE ways_nodes(id, node_id, position)



The database consists of 5 tables (nodes, nodes_tags, ways, ways_tags, and ways_nodes), created from CSV data which was generated in the previous section.

### Sort cities by count, descending

In [52]:
query = '''SELECT tags.value, COUNT(*) as count
            FROM (SELECT * FROM nodes_tags UNION ALL
                    SELECT * FROM ways_tags) tags
            WHERE tags.key LIKE "city"
            GROUP BY tags.value
            ORDER BY count DESC
            LIMIT 15;'''
print_results(db_name, query)

Milwaukee	2233	
Racine	638	
Mount Pleasant	238	
Burlington	41	
Sturtevant	35	
Caledonia	32	
Waukesha	32	
MIlwaukee	23	
Brown Deer	21	
West Allis	21	
Hartland	20	
Wauwatosa	20	
Brookfield	19	
Cedarburg	19	
Glendale	19	


While a large portion of the nodes are located in Milwaukee, there are a few which contain cities in the Milwaukee metropolitan area. Some cities (like Racine and Mount Pleasant) are actually a fairly long distance away from Milwaukee (25 miles between city centers for Milwaukee -> Racine).

Postal code data seems to be okay, with all listed postal codes starting with "53XXX". This indicates all postal codes are within some vicinity of Milwaukee

In [53]:
# Show all postal codes listed in database to ensure replacement function worked correctly

query = """SELECT tags.value, COUNT(*) as count
FROM (SELECT * FROM nodes_tags
		UNION ALL
		SELECT * FROM ways_tags) tags
WHERE tags.key = 'postcode'
GROUP BY tags.value
ORDER BY count DESC
LIMIT 20;"""

print_results(db_name, query)

53202	1372	
53212	339	
53203	236	
53233	132	
53205	96	
53211	91	
53204	67	
53215	47	
53027	45	
53105	45	
53216	28	
53222	25	
53186	23	
53223	23	
53029	22	
53012	20	
53207	20	
53402	19	
53213	17	
53217	16	


### Data Overview

#### File sizes
    milwaukee_wisconsin.osm..... 188.8 MB
    milwaukee_osm.db............ 256.2 MB
    nodes.csv...................  70.5 MB
    nodes_tags.csv..............   2.4 MB
    ways.csv....................   5.7 MB
    ways_tags.csv...............  15.5 MB
    ways_nodes.csv..............  24.6 MB

#### Number of Nodes and Ways

In [54]:
query = "SELECT COUNT(*) FROM nodes;"
print 'Number of Nodes:'
print '----------------'
print_results(db_name, query)
print 'vs.', tags['node'], '(From iterparse method)'

print 

query = "SELECT COUNT(*) FROM ways;"
print 'Number of Ways:'
print '---------------'
print_results(db_name, query)
print 'vs.', tags['way'], '(From iterparse method)'

Number of Nodes:
----------------
840764	
vs. 840763 (From iterparse method)

Number of Ways:
---------------
94039	
vs. 94038 (From iterparse method)


#### Number of Unique Users

In [55]:
query = """SELECT COUNT(DISTINCT(e.uid))
FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e;"""
print 'Number of unique users'
print '----------------------'
print_results(db_name, query)

Number of unique users
----------------------
641	


#### Top 10 contributing users

In [56]:
query = '''SELECT e.user, COUNT(*) as num
FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e
GROUP BY e.user
ORDER BY num DESC
LIMIT 10;'''
print 'Top 10 contributing users:'
print '--------------------------'
print_results(db_name, query)

Top 10 contributing users:
--------------------------
woodpeck_fixbot	178980	
shuui	136230	
ItalianMustache	118347	
reschultzed	61072	
bbauter	29048	
Gary Cox	26439	
hogrod	25616	
iandees	25396	
Mulad	22393	
TIGERcnl	21652	


#### Number of users appearing only once (having 1 post)

In [57]:
query = ''' SELECT COUNT(*)
            FROM
                (SELECT e.user, COUNT(*) as num
                FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e
                GROUP BY e.user
                HAVING num=1) u;'''
print 'Number of users appearing only once:'
print '------------------------------------'
print_results(db_name, query)

Number of users appearing only once:
------------------------------------
113	


### Data Exploration
#### Top 10 Appearing Amenities

In [58]:
query = '''SELECT value, COUNT(*) as num
            FROM nodes_tags
            WHERE key="amenity"
            GROUP BY value
            ORDER BY num DESC
            LIMIT 10;'''
print 'Top 10 appearing amenities:'
print '---------------------------'
print_results(db_name, query)

Top 10 appearing amenities:
---------------------------
parking_entrance	896	
school	627	
restaurant	299	
bench	285	
fast_food	144	
grave_yard	129	
fuel	116	
parking	86	
cafe	83	
bicycle_parking	68	


#### Number of coffee shop / cafe locations, grouped by company (very important to me!)
It should be noted that Starbucks appears multiple times in this list (as well as some other cafes, such as Colectivo). This database could benefit from some data cleaning to standardize the names of these companies to a single spelling, to avoid redundant list items like those found below.

In [59]:
query = '''SELECT nodes_tags.value, COUNT(*) as num
            FROM nodes_tags, (
                 SELECT *
                   FROM nodes_tags 
                   WHERE key="amenity" AND value = "cafe") as cafe
            WHERE nodes_tags.id = cafe.id AND nodes_tags.key="name"
            GROUP BY nodes_tags.value
            ORDER BY num DESC
            LIMIT 10'''
print_results(db_name, query)

Starbucks	9	
Starbucks Coffee	3	
Stone Creek Coffee	3	
Colectivo Coffee	2	
Dunkin' Donuts	2	
Starbuck's	2	
2894 On Main	1	
600 East Cafe	1	
8th Note Coffee House	1	
Alderaan Coffee	1	


#### Most popular cuisines in the city

In [61]:
query = '''SELECT nodes_tags.value, COUNT(*) as num
            FROM nodes_tags
                JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='restaurant') i
                ON nodes_tags.id = i.id
            WHERE nodes_tags.key = 'cuisine'
            GROUP BY nodes_tags.value
            ORDER BY num DESC
            LIMIT 25;'''
print_results(db_name, query)

american	26	
pizza	25	
italian	16	
sandwich	15	
chinese	13	
burger	10	
american_new	7	
mexican	7	
japanese	6	
regional	5	
greek	4	
german	3	
seafood	3	
steak_house	3	
vietnamese	3	
asian	2	
bagel	2	
diner	2	
ethiopian	2	
indian	2	
irish	2	
korean	2	
new_american	2	
new_orleans	2	
world	2	


This is great! Although I wish there were more, there are 2 Korean restaurants in the city and surrounding area! I'm curious to find out the names of these restaurants so I can look into them further.

In [64]:
# Name of all Korean restaurants
query = '''SELECT restaurant.value, nodes_tags.value
            FROM nodes_tags, (
                 SELECT *
                   FROM nodes_tags 
                   WHERE key="cuisine" AND (value = "korean" OR (value = "new_orleans"))) as restaurant
            WHERE nodes_tags.id = restaurant.id AND nodes_tags.key="name"'''
print_results(db_name, query)

korean	Seoul Korean Restaurant	
korean	Stone Bowl Grill	
new_orleans	Evolution Gastro Pong	
new_orleans	The Brass Alley	


They both offer Korean BBQ!! And have pretty good reviews on Yelp!

#### Most popular grocery stores
Pick n' Save seems to be the winner here, but there are a few local markets available, and of course a couple of Walmart's

In [65]:
query = '''SELECT nodes_tags.value, COUNT(*) as num
            FROM nodes_tags, (
                 SELECT *
                   FROM nodes_tags 
                   WHERE key="shop" AND (value = "supermarket")) as grocery
            WHERE nodes_tags.id = grocery.id AND nodes_tags.key="name"
            GROUP BY nodes_tags.value
            ORDER BY num DESC
            LIMIT 10;'''
print_results(db_name, query)

Aldi	11	
Piggly Wiggly	8	
Pick n' Save	7	
Pick 'n' Save	3	
Sentry Foods	3	
Walmart Supercenter	3	
Outpost Natural Foods	2	
Pick 'n Save	2	
Pick'n Save	2	
Sendik's	2	


## Conclusion


In general, the data did require some cleaning for consistency, but appeared to be accurate with respects to all nodes being within the Milwaukee metropolitan area. I would suggest further cleaning in the names of amenities to allow more accurate grouping with SQL queries (for example: "StarBucks" vs. "Starbucks", or "Pick N Save" vs. "Pick n Save"). 

This task, however, is quite labor intensive. While it is something that could be done to an extent programmatically using regular expressions, it will most likely also require a significant amount of manual user input to be performed completely.

## References
Sample Project - https://gist.github.com/carlward/54ec1c91b62a5f911c42#file-sample_project-md